In [1]:
from GPyOpt.acquisitions.GES import AcquisitionGES

In [2]:
#! /usr/bin/env python

%matplotlib inline

import os,sys
import os.path as osp

import argparse
import os, tempfile, glob, shutil
from datetime import datetime

import GPy
import GPyOpt
import subprocess as sub
import numpy as np
import scipy.spatial
import math
import matplotlib.pyplot as plt
from numpy import linalg as LA

from scipy import optimize
import time
import socket

import rospy
from geometry_msgs.msg import PoseStamped
from std_msgs.msg import Int16

import geometry_msgs.msg
import random
import tf 

from scipy.stats import gaussian_kde
# from sklearn.neighbors import NearestNeighbors

from GPyOpt.acquisitions.EI import AcquisitionEI
from GPyOpt.acquisitions.MPI import AcquisitionMPI

from numpy.random import seed

import dill


global g_blender_executable_path,  blank_file, render_code, robot_file, model_file, table_file, temp_dirname, epsilon, \
       filename, mass, friction, temp_dirname, location_gt, rotation_gt, obj_start_config_array, hand_start_config_array, \
       obj_end_config_array, hand_end_config_array, id_of_sample, train_file, init_pose_file, final_pose_file

epsilon = 1e-6

##############################################
if os.environ.get('BLENDER_PATH') == None:
    print("Please set BLENDER_PATH in bashrc!")
    sys.exit()
g_blender_executable_path = os.environ['BLENDER_PATH']

if os.environ.get('PHYSICS_SIMULATOR_PATH') == None:
    print("Please set PHYSICS_SIMULATOR_PATH in bashrc!")
    sys.exit()

g_repo_path = os.environ['PHYSICS_SIMULATOR_PATH']

# Input Pose File path
init_pose_file = g_repo_path + '/tmp/gp_sim_init_pose.txt'
best_action_file = g_repo_path + '/tmp/gp_sim_best_actiom.txt'
final_pose_file = g_repo_path + '/tmp/gp_sim_final_pose.txt'

g_blank_blend_file_path = g_repo_path + '/blank.blend'
obj_model_folder = g_repo_path+'/obj_models/'
table_model_folder = g_repo_path+'/table_models/'
model_name = 'expo_dry_erase_board_eraser'
robot_name = 'reflex'
table_name = 'table'


blank_file = osp.join(g_blank_blend_file_path)
model_file = osp.join(obj_model_folder, model_name+ '.obj')
robot_file = osp.join(obj_model_folder, robot_name+ '.obj')
table_file = osp.join(table_model_folder, table_name+ '.obj')
temp_dirname = '/home/zhusj/repos/blender_scene_generator/test'  

In [3]:
def send_to_sim(mass, friction, action, x_offset):
    
    obj_state = [-0.107687, -0.535722, 0.791, 0.0103682, 0.941487, -0.0294104, 0.335604] #q:wxyz #np.random.uniform(-1, 1, size=(2,))
    obj_goal = [obj_state[0]+1.0, obj_state[1], obj_state[2]]
    action = 1
    
    with open(init_pose_file, "w+") as file:
        file.write("%f %f %f %f %f %f %f %f %f %f %f %f \n" % \
                   (mass, friction, obj_state[0], obj_state[1], obj_state[3], obj_state[4], \
                    obj_state[5], obj_state[5], action, x_offset, obj_goal[0], obj_goal[1]))


    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

    # Bind the socket to the port
    server_address = ('localhost', 10000)
    sock.bind(server_address)

    # Listen for incoming connections
    sock.listen(1)

     # Create a TCP/IP socket
    sock2 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # Connect the socket to the port where the server is listening
    serv_address = ('localhost', 50008)
    sock2.connect(serv_address)
    sock2.sendall(str(action))
    sock2.close()

    #     location, rotation = run_physics_engine(filename, id_of_sample, mass, friction, obj_start_config)

    # Wait for a connection
    #     print('waiting for a connection')
    connection, client_address = sock.accept()

    # Receive the data in small chunks and retransmit it
    data = connection.recv(16)
    #     print('received %s' % data)
    connection.close()

    sock.close()

    with open(final_pose_file, "r") as file:
        for line in file:
            vals = line.split()
            objx = vals[0]
            objy = vals[1]
            rx =vals[3]
            ry = vals[4]
            rz = vals[5]
            rw = vals[6]

    location =[]
    location.append([float(objx), float(objy)])
    location = np.asarray(location)

    rotation = []
    rotation.append([float(rx),float(ry),float(rz),float(rw)])
    rotation = np.asarray(rotation)
    
    return location, rotation

In [4]:
def myf(X):
#     start_time1 = time.time()
    mass = X[0][0]
    friction = X[0][1]

#     print 'mass & friction: ', mass, friction
    y_new = 0
    
#    for count in xrange(0, number_of_samples):
    action =  np.random.uniform(0.8, 1.2)
    x_offset = np.random.uniform(-0.05, 0.05)
    y_offset = np.random.uniform(-0.02, 0.02)

     
    location, rotation = send_to_sim(mass, friction, action, x_offset)

    mass_gt = 0.24351668  
    friction_gt = 0.05537921
    location_gt, rotation_gt = send_to_sim(mass_gt, friction_gt, action, x_offset)

#     print "rotation:", rotation, rotation_gt

    p_dist = np.sqrt(np.sum(((location-location_gt)**2)))
    
#    q_dist = np.abs(1 - np.sum(np.multiply(rotation, rotation_gt)**2))
#     q_dist = 1 - np.sum(np.multiply(rotation, rotation_gt)**2)        

#     print 'dist: ', p_dist, q_dist/10
    y_new = p_dist# + (q_dist/10)

#     elapsed_time = time.time() - start_time1
#     print "elapsed_time: ", elapsed_time

#     print "y_new: ", y_new
    return y_new

In [12]:
def compute_success_probability(myBopt, action, new_fps):
    

    
    bounds = myBopt.acquisition.space.get_bounds()
    acquisition_function = myBopt.acquisition.acquisition_function

    size = 50

    X1 = np.linspace(bounds[0][0], bounds[0][1], size)
    X2 = np.linspace(bounds[1][0], bounds[1][1], size)
    x1, x2 = np.meshgrid(X1, X2)
    X = np.hstack((x1.reshape(size*size,1),x2.reshape(size*size,1)))
    
    x_opt = np.reshape(myBopt.x_opt, (-1, 2))
    X_new = np.concatenate((X, x_opt), axis =0)
    
    acqu = acquisition_function(X_new)
    print 'acqu[-1]: ', acqu[-1]
    
    acqu_normalized = (-acqu - min(-acqu))/(max(-acqu - min(-acqu)))    
    print 'acqu_normalized[-1]: ', acqu_normalized[-1]
    temp = np.argpartition(-acqu_normalized, 5,axis=0)
    result_args = temp[:5]
    temp = np.partition(-acqu_normalized, 5,axis=0)
    probability = -temp[:5]
    
    print 'probability: ', probability
    probability_normalized = probability /np.sum(probability) 
    probability_normalized = np.squeeze(probability_normalized)
    print 'probability_normalized: ', probability_normalized

    sampled_X = X[result_args,:]
    
    sampled_X = np.squeeze(sampled_X)
    sampled_X = np.concatenate((sampled_X, x_opt), axis=0)
    sampled_dist = np.zeros((sampled_X.shape[0],))
    
    for i in xrange(0, sampled_X.shape[0]):
        obj_state = [-0.107687, -0.535722, 0.791, 0.0103682, 0.941487, -0.0294104, 0.335604] #q:wxyz #np.random.uniform(-1, 1, size=(2,))
        obj_goal = [obj_state[0]+1.0, obj_state[1], obj_state[2]]
        mass = sampled_X[i,0]
        friction = sampled_X[i,1]
        print 'mass, friction: ', mass, friction
        
        with open(init_pose_file, "w+") as file:
            file.write("%f %f %f %f %f %f %f %f %f %f %f %f \n" % \
                       (mass, friction, obj_state[0], obj_state[1], obj_state[3], obj_state[4], \
                        obj_state[5], obj_state[5], action, new_fps, obj_goal[0], obj_goal[1]))


        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

        # Bind the socket to the port
        server_address = ('localhost', 10101)
        sock.bind(server_address)

        # Listen for incoming connections
        sock.listen(1)

         # Create a TCP/IP socket
        sock2 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

        # Connect the socket to the port where the server is listening
        serv_address = ('localhost', 50101)
        sock2.connect(serv_address)
        sock2.sendall(str(action))
        sock2.close()

        #     location, rotation = run_physics_engine(filename, id_of_sample, mass, friction, obj_start_config)

        # Wait for a connection
        #     print('waiting for a connection')
        connection, client_address = sock.accept()

        # Receive the data in small chunks and retransmit it
        data = connection.recv(16)
        #     print('received %s' % data)
        connection.close()

        sock.close()

        with open(final_pose_file, "r") as file:
            for line in file:
                vals = line.split()
                dist = vals[0]
        sampled_dist[i] = dist
        print 'dist:', dist
    
    current_optimal_dist = sampled_dist[-1]
#     print 'sampled_dist: ', sampled_dist.shape
#     print 'probability_normalized: ', probability_normalized.shape
    weighted_dist = np.sum(np.multiply(sampled_dist[:-1],probability_normalized))
    
    print 'weighted_dist: ', weighted_dist
    print 'current_optimal_dist: ', current_optimal_dist
    ratio = current_optimal_dist/weighted_dist
    print 'ratio: ', ratio
        
    return ratio

In [13]:
BO_result = []

space = GPyOpt.Design_space(space =[{'name': 'mass', 'type': 'continuous', 'domain': (0,1)},
                                    {'name': 'friction', 'type': 'continuous', 'domain': (0,0.5)}])
mass_max = 0.9
friction_max = 0.9
total_trial = 50
max_number_of_samples = 50
step = 1

num_of_trials = 1
total_iters = 3
time_result = np.zeros((num_of_trials,total_iters))
dist_result = np.zeros((num_of_trials,total_iters))


bounds = [{'name': 'mass', 'type': 'continuous', 'domain': (0.1,mass_max)},
        {'name': 'friction', 'type': 'continuous', 'domain': (1e-3,friction_max)}]
num_cores = 10

model = GPyOpt.models.GPModel(optimize_restarts=5,verbose=False, normalize_Y = False)

aquisition_optimizer = GPyOpt.optimization.AcquisitionOptimizer(space)

from numpy.random import seed # fixed seed
# seed(123456)

acquisition = AcquisitionMPI(model, space, optimizer=aquisition_optimizer) 

evaluator = GPyOpt.core.evaluators.Sequential(acquisition)

max_time = 0
max_iter = 0
eps = 1e-2
unit_time = 5
unit_iter = 2

eps =0.05
kwargs = dict(model_optimize_restarts=True)


ratio_threshold = [0.5, 0.7, 0.9]

for trial in range(0,num_of_trials):
    for iter in range(0,total_iters):
        
        k = GPy.kern.Matern32(input_dim=2, ARD=True)
        myBopt = GPyOpt.methods.BayesianOptimization(myf,
                                                domain=bounds,
                                                kernel = k, acquisition_type ='GES'
                                               )
        start_time = time.time()
        iter_count = 0
        ratio = 0
        print "starting trial ", trial, ",　iter ", iter, ", ratio_threshold:", \
                ratio_threshold[iter], ",　at ", datetime.now()

        while ratio < ratio_threshold[iter]:
            
            max_iter = unit_iter# *(iter+1)

#             print "max_iter: ", max_iter

            myBopt.run_optimization(max_iter=max_iter)
            iter_count += max_iter
            
#             print "optima!!!!!!!!!!!!!!!!: ", myBopt.x_opt, myBopt.fx_opt, " in ", myBopt.cum_time

            optimal_mass = myBopt.x_opt[0]
            optimal_friction = myBopt.x_opt[1]

            obj_state = [-0.107687, -0.535722, 0.791, 0.0103682, 0.941487, -0.0294104, 0.335604] #q:wxyz #np.random.uniform(-1, 1, size=(2,))
            obj_goal = [obj_state[0]+1.0, obj_state[1], obj_state[2]]
            action = 1

            with open(init_pose_file, "w+") as file:
                file.write("%f %f %f %f %f %f %f %f %f %f %f %f \n" % \
                           (optimal_mass, optimal_friction, obj_state[0], obj_state[1], obj_state[3], obj_state[4], \
                            obj_state[5], obj_state[6], action, obj_state[0], obj_goal[0], obj_goal[1]))

            # Create a TCP/IP socket
            sock2 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock2.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

            # Connect the socket to the port where the server is listening
            serv_address = ('localhost', 50099)
            #print('waiting for a connection')
            sock2.connect(serv_address)
            #print('Connected')
            sock2.sendall(b'hi')
            sock2.close()   

            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
            # Bind the socket to the port
            server_address = ('localhost', 10099)
            sock.bind(server_address)

            # Listen for incoming connections
            sock.listen(1)        
            connection, client_address = sock.accept()

            # Receive the data in small chunks and retransmit it
            data = connection.recv(16)
            connection.close()
            sock.close()             

            with open(best_action_file, "r") as file:
                for line in file:
                    vals = line.split()
                    action = float(vals[0])
                    new_fps = float(vals[1])

            print "action, new_fps: ", action, new_fps
            elapsed_time = time.time() - start_time

            ratio = compute_success_probability(myBopt, action, new_fps)

            if ratio > ratio_threshold[iter]:
                with open(final_pose_file, "r") as file:
                    for line in file:
                        vals = line.split()
                        dist = float(vals[0])

                elapsed_time = time.time() - start_time

                print "\n dist: ", dist, "in ", elapsed_time, "with ", iter_count, "iters \n"
            else:  
                if myBopt.cum_time == 0:
                    print "Reinitializing GP, myBopt.X.shape: ", myBopt.X.shape
        #             k = GPy.kern.Matern32(input_dim=2, ARD=True)
                    myBopt = GPyOpt.methods.BayesianOptimization(myf,
                                                             domain=bounds,
                                                             kernel = myBopt.kernel,X=myBopt.X[:-max_iter,:], Y=myBopt.Y[:-max_iter,:]
                                                            )
                    print "Reinitializing GP, myBopt.X.shape: ", myBopt.X.shape

        time_result[trial, iter] = elapsed_time
        dist_result[trial, iter] = dist
        BO_result.append([myBopt, max_iter])
        
print "Finished!!!"

starting trial  0 ,　iter  0 , ratio_threshold: 0.5 ,　at  2017-03-01 18:36:14.790936
action, new_fps:  20.0 80.0
acqu[-1]:  [-0.36726134]
acqu_normalized[-1]:  [ 1.]
probability:  [[ 0.99913165]
 [ 1.        ]
 [ 0.99913164]
 [ 0.99913163]
 [ 0.99913162]]
probability_normalized:  [ 0.19996524  0.20013903  0.19996524  0.19996524  0.19996524]


IndexError: index 2500 is out of bounds for axis 0 with size 2500